Performance Report ETL

In [ ]:
# Import needed libriaries
import pandas as pd

In [2]:
# Load raw data from Excel file
outage_atc = pd.read_excel('C:\\Users\\mayow\\OneDrive\\Documents\\01 Personal Documents\\03 Projects Data Analytics\\01 Telecommunications Site Outages Analysis and Reporting\\assets v1.2\\datasets\\Performance Data.xlsx', sheet_name='Outage')
performance_atc = pd.read_excel('C:\\Users\\mayow\\OneDrive\\Documents\\01 Personal Documents\\03 Projects Data Analytics\\01 Telecommunications Site Outages Analysis and Reporting\\assets v1.2\\datasets\\Performance Data.xlsx', sheet_name='2023 - 2024 PA Trends')

c:\Users\mayow\anaconda3\envs\mayowa-data-analytics\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [ ]:
# Select only needed Columns from Outage report
outage_atc = outage_atc[['Issue Number',
       'Airtel Site ID', 'Alarm Start Time', 'DURATION', 'MTTR',
       'Root Cause', 'Category', 'Sub-Cat 1',
       'Sub-Cat 2', 'Sub-Cat 3', 'Underlying Cause', 'AIRTEL CLASS']]

In [ ]:
# Functions to:
# - Create a column in outage table and fill it with specific text based on another column
# - Check if a column contains date format
def classify_rca(row):
    if pd.isnull(row['Underlying Cause']):
        return pd.Series(['Other Issues', 'Others', 'Others', 'Others'])  # Handle NaN values appropriately
    
    rca = row['Underlying Cause'].lower()
    
    if 'access' in rca:
        if 'unpaid security salary' in rca:
            return pd.Series(['Access Issue', 'Unpaid Security Salary', 'Guard restriction', 'Others'])
        elif 'landlord' in rca:
            return pd.Series(['Access Issue', 'Landlord Restriction', 'Landlord', 'Others'])
        elif 'restriction' in rca:
            return pd.Series(['Access Issue', 'Landlord Restriction', 'Landlord restriction', 'Others'])
        else:
            return pd.Series(['Access Issue', 'Landlord Restriction', 'Landlord Challenge', 'Others'])
    elif any(x in rca for x in ['fuel', 'diesel', 'separator']):
        if any(x in rca for x in ['low', 'outage', 'no']):
            return pd.Series(['Generator', 'Mechanical', 'Fuel', 'Low Diesel'])
        elif any(x in rca for x in ['contaminated', 'spill']):
            return pd.Series(['Generator', 'Mechanical', 'Fuel', 'Contaminated Fuel'])
        elif 'priming pump' in rca:
            return pd.Series(['Generator', 'Mechanical', 'Fuel', 'Faulty Priming Pump'])
        else:
            return pd.Series(['Generator', 'Mechanical', 'Fuel', 'Others'])
    elif 'hybrid' in rca:
        return pd.Series(['Hybrid Issues', 'Others', 'Others', 'Others'])
    elif 'compression' in rca:
        return pd.Series(['Generator', 'Mechanical', 'Engine', 'Loss Compression'])
    elif 'rim' in rca:
        return pd.Series(['Generator', 'Mechanical', 'Engine', 'Hand Primer'])
    elif 'attery' in rca:
        return pd.Series(['Other Equipment', 'Battery', 'Starter Batteries', 'Starter Batteries'])
    elif 'low engine oil' in rca:
        return pd.Series(['Generator', 'Mechanical', 'Engine', 'Low Engine Oil'])
    elif 'overheating' in rca:
        return pd.Series(['Generator', 'Mechanical', 'Engine', 'Overheating'])
    elif 'cable' in rca:
        return pd.Series(['Other Equipment', 'Electrical', 'Wiring', 'Cable'])
    elif 'oil leak' in rca:
        return pd.Series(['Generator', 'Mechanical', 'Engine', 'Oil Leak'])
    elif 'cylinder head gasket' in rca:
        return pd.Series(['Generator', 'Mechanical', 'Engine', 'Cylinder Head Gasket Fault'])
    elif 'injector pump' in rca:
        return pd.Series(['Generator', 'Mechanical', 'Injector Pump', 'Faulty Injector Pump'])
    elif 'coolant' in rca:
        return pd.Series(['Generator', 'Mechanical', 'Radiator', 'DG Coolant Low'])
    elif 'radiator overheating' in rca:
        return pd.Series(['Generator', 'Mechanical', 'Radiator', 'DG Overheating'])
    elif 'radiator leakages' in rca:
        return pd.Series(['Generator', 'Mechanical', 'Radiator', 'Radiator Leakages'])
    elif 'e-pump' in rca:
        return pd.Series(['Generator', 'Electrical', 'E-pump', 'Faulty Electric Pump'])
    elif 'hard start' in rca:
        return pd.Series(['Generator', 'Electrical', 'Battery', 'Hard Start'])
    elif 'battery failure' in rca:
        return pd.Series(['Generator', 'Electrical', 'Battery', 'Battery Failure'])
    elif 'voltage fluctuation' in rca or 'AVR' in rca:
        return pd.Series(['Generator', 'Electrical', 'AVR', 'Voltage Fluctuation'])
    elif 'hargin' in rca:
        return pd.Series(['Generator', 'Electrical', 'Charging Alternator', 'Failure'])
    elif 'tripped circuit breaker' in rca or 'breaker failure' in rca or 'breaker' in rca:
        return pd.Series(['Generator', 'Electrical', 'Breaker', 'Tripped Circuit Breaker'])
    elif 'control fault' in rca:
        return pd.Series(['Generator', 'Electrical', 'Control Panel', 'Control Fault'])
    elif 'alternator fault' in rca:
        return pd.Series(['Generator', 'Electrical', 'Alternator', 'Alternator Fault'])
    elif 'wiring fault' in rca:
        return pd.Series(['Other Equipment', 'Electrical', 'Wiring', 'Wiring Fault'])
    elif 'transformer fault' in rca:
        return pd.Series(['Generator', 'Electrical', 'Transformer', 'Transformer Fault'])
    elif 'faulty libs' in rca:
        return pd.Series(['Other Equipment', 'Battery', 'Faulty LIBs', 'Others'])
    elif 'faulty inverter' in rca:
        return pd.Series(['Other Equipment', 'Inverter', 'Faulty Inverter', 'Others'])
    elif 'faulty rectifier' in rca:
        return pd.Series(['Other Equipment', 'Rectifier', 'Faulty Rectifier', 'Others'])
    elif 'faulty backup batteries' in rca:
        return pd.Series(['Other Equipment', 'Back up Battery', 'Faulty Backup Batteries', 'Others'])
    elif 'generator theft' in rca:
        return pd.Series(['Theft', 'Generator Theft', 'Generator components theft', 'Others'])
    elif 'cable theft' in rca or 'ac/dc cable theft' in rca:
        return pd.Series(['Theft', 'Cable Theft', 'DC Cable Theft', 'Others'])
    elif 'site vandalism' in rca:
        return pd.Series(['Theft', 'Site Vandalism', 'Site Vandalized', 'Others'])
    elif any(x in rca for x in ['servicing', 'plan', 'activi']):
        if 'maintenance' in rca:
            return pd.Series(['Planned Activities', 'Preventative Maintenance', 'Others', 'Others'])
        elif any(x in rca for x in ['installation', 'swap']):
            return pd.Series(['Planned Activities', 'Generator Installation/Swap', 'Others', 'Others'])
        else:
            return pd.Series(['Planned Activities', 'Other', 'Others', 'Others'])
    elif 'grid' in rca:
        return pd.Series(['Grid Issues', 'Others', 'Others', 'Others'])
    elif 'rectif' in rca:
        return pd.Series(['Other Equipment', 'Rectifier', 'Others', 'Others'])
    elif 'theft' in rca or 'vanda' in rca:
        if 'cable' in rca:
            return pd.Series(['Theft', 'Cable Theft', 'Others', 'Others'])
        else:
            return pd.Series(['Theft', 'Generator Theft', 'Others', 'Others'])
    elif 'ats' in rca:
        return pd.Series(['Generator', 'Electrical', 'ATS', 'ATS Fault'])
    else:
        return pd.Series(['Other Issues', 'Others', 'Others', 'Others'])

def is_date(string):
    try:
        pd.to_datetime(string)
        return True
    except ValueError:
        return False

In [ ]:

# Create Key column in outage table for ensure unique entries
outage_atc['key'] = outage_atc['Issue Number'].astype(str) + '_' + outage_atc['Alarm Start Time'].astype(str)
outage_atc['incident date'] = outage_atc['Alarm Start Time'].dt.date.astype('datetime64[ns]')

# Use the previously created function to check date columns only and add to "Anchor tenant" column
date_columns_atc = [col for col in performance_atc.columns if is_date(col)]
columns_to_keep_atc = ['Anchor Tenant'] + date_columns_atc

atc = performance_atc[columns_to_keep_atc]

In [ ]:
# unpivot with melt function
melted_atc = pd.melt(atc, id_vars=['Anchor Tenant'], var_name='Date', value_name='Performance')

In [ ]:
# Drop entries with empty values
melted_atc = melted_atc.dropna(subset=['Performance'])

melted_atc['Date'] = pd.to_datetime(melted_atc['Date'])

print('----ATC PA----')
print(melted_atc.tail(2))
print('----------------------------------------------')
print('----ATC Outage----')
print(outage_atc['Alarm Start Time'].tail(2))

----ATC PA----
       Anchor Tenant       Date  Performance
863377        AB1605 2024-11-24        100.0
863378        AB1618 2024-11-24        100.0
----------------------------------------------
----ATC Outage----
609   2024-11-24
610   2024-11-24
Name: Alarm Start Time, dtype: datetime64[ns]


In [ ]:
# load cleaned data and make sure some columns are datetime format
outage_database_atc = pd.read_csv('C:\\Users\\mayow\\OneDrive\\Documents\\01 Personal Documents\\03 Projects Data Analytics\\01 Telecommunications Site Outages Analysis and Reporting\\assets v1.2\\datasets\\Cleaned Datasets\\atc_outages.csv')
outage_database_atc['incident date'] = pd.to_datetime(outage_database_atc['incident date'])
outage_database_atc['Alarm Start Time'] = pd.to_datetime(outage_database_atc['Alarm Start Time'])

outage_database_atc['incident date'].tail(1)


15094   2024-11-24
Name: incident date, dtype: datetime64[ns]

In [ ]:
# Use key to avoid appending duplicates when add new entries

existing_keys = set(outage_database_atc['key'])
new_outages = outage_atc[~outage_atc['key'].isin(existing_keys)]

if not new_outages.empty:
    new_outages.to_csv('atc_outages.csv', mode='a', header=False, index=False)
    
new_outages.info()

<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Issue Number      0 non-null      int64         
 1   Airtel Site ID    0 non-null      object        
 2   Alarm Start Time  0 non-null      datetime64[ns]
 3   DURATION          0 non-null      float64       
 4   MTTR              0 non-null      float64       
 5   Root Cause        0 non-null      object        
 6   Category          0 non-null      object        
 7   Sub-Cat 1         0 non-null      object        
 8   Sub-Cat 2         0 non-null      object        
 9   Sub-Cat 3         0 non-null      object        
 10  Underlying Cause  0 non-null      object        
 11  AIRTEL CLASS      0 non-null      object        
 12  key               0 non-null      object        
 13  incident date     0 non-null      datetime64[ns]
dtypes: datetime64[ns](2), float64(2), int64(1),

In [ ]:
# Create new columns with the classify rca function

outage_atc = pd.read_csv('C:\\Users\\mayow\\OneDrive\\Documents\\01 Personal Documents\\03 Projects Data Analytics\\01 Telecommunications Site Outages Analysis and Reporting\\assets v1.2\\datasets\\Cleaned Datasets\\atc_outages.csv')
outage_atc[['Cat', 'S-Cat1', 'S-Cat2', 'S-Cat3']] = outage_atc.apply(classify_rca, axis=1)

outage_atc['incident date'] = pd.to_datetime(outage_atc['incident date'])
outage_atc['Alarm Start Time'] = pd.to_datetime(outage_atc['Alarm Start Time'])

In [ ]:
# Save to CSV

outage_atc.to_csv('C:\\Users\\mayow\\OneDrive\\Documents\\01 Personal Documents\\03 Projects Data Analytics\\01 Telecommunications Site Outages Analysis and Reporting\\assets v1.2\\datasets\\Cleaned Datasets\\atc_outages.csv', index=False)

melted_atc.to_csv('C:\\Users\\mayow\\OneDrive\\Documents\\01 Personal Documents\\03 Projects Data Analytics\\01 Telecommunications Site Outages Analysis and Reporting\\assets v1.2\\datasets\\Cleaned Datasets\\atc_performance.csv', index=False)

In [14]:
print('----------------------------------------------------------------')
melted_atc.info()
print('----------------------------------------------------------------')

----------------------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
Index: 827686 entries, 0 to 863378
Data columns (total 3 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   Anchor Tenant  827686 non-null  object        
 1   Date           827686 non-null  datetime64[ns]
 2   Performance    827686 non-null  float64       
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 25.3+ MB
----------------------------------------------------------------
